In [1]:
%%html
<style>
h1, h2, h3, h4, h5 {
    color: darkblue;
    font-weight: bold !important;
}
h2 {
    border-bottom: 8px solid darkblue !important;
    padding-bottom: 8px;
}
h3 {
    border-bottom: 2px solid darkblue !important;
    padding-bottom: 6px;
}
.info, .success, .warning, .error {
    border: 1px solid;
    margin: 10px 0px;
    padding:15px 10px;
}
.info {
    color: #00529b;
    background-color: #bde5f8;
}
.success {
    color: #4f8a10;
    background-color: #dff2bf;
}
.warning {
    color: #9f6000;
    background-color: #FEEFB3;
}
.error {
    color: #D8000C;
    background-color: #FFBABA;
}
.language-bash {
    font-weight: 900;
}
.ex {
    font-weight: 900;
    color: rgba(27,27,255,0.87) !important;
}
.mn {
    font-family: Menlo, Consolas, "DejaVu Sans Mono", monospace
}
table {
    margin-left: 0 !important;}
</style>

# 4.4 Excel with Pandas

<span class='ex'>Example: <span class='mn'>pandas.read_excel()</span></span>

In [50]:
import pandas as pd
import numpy as np

df  = pd.read_excel('database.xlsx')  # df means DataFrame
df

,Area,SwitchIP,Hostname,Function,Netmask,Username,Password,Devicetype,Serialno,WarrantyExpiring,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,A,192.168.99.2,S01,Test,255.255.255.0,admin,class,cisco-ios,123879.0,2019-Dec-23,NaN,Failed on 2019-Oct-11,NaN
1,A,192.168.99.3,S01,Web,255.255.255.0,admin,class,cisco-ios,23131.0,2020-Mar-02,NaN,To be shutdown,NaN
2,B,192.168.99.8,S01,Database,255.255.255.0,admin,class,cisco-ios,313131.0,NaN,NaN,NaN,NaN
3,C,192.168.99.13,S01,Devel,255.255.255.0,admin,class,cisco-ios,NaN,2022-Feb-22,NaN,NaN,4000.0
4,C,192.168.99.14,S01,Devel,255.255.255.0,admin,class,cisco-ios,NaN,2020-Mar-02,NaN,NaN,NaN


There are unwanted columns with names `Unnamed: *`

Let's retrieve the list of column labels using `df.columns`, and the delete unwanted columns.

In [51]:
import re
pat = re.compile('Unnamed.*')
cols = list(df.columns)
print(cols, '\n')

usecols = [col for col in cols if not re.match(pat, col)]
print(usecols)

['Area', 'SwitchIP', 'Hostname', 'Function', 'Netmask', 'Username', 'Password', 'Devicetype', 'Serialno', 'WarrantyExpiring', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12'] 

['Area', 'SwitchIP', 'Hostname', 'Function', 'Netmask', 'Username', 'Password', 'Devicetype', 'Serialno', 'WarrantyExpiring']


If a read-in column contains empty cell, pandas returns `NaN`.  If we do not use that column, we could just leave as it is.

However, if we need to process data using that column, it is better we replace empty cell with other values.

Let's re-read the Excel file with `usecols` only and replace `NaN` with `''`.

In [52]:
def myconvert(value):
    if value.upper() == 'TRUE':
        return str(1)
    else:
        return str(0)

df  = pd.read_excel(
    'database.xlsx',
    usecols=usecols,
    converters={
        "Serialno":         lambda x: '-' if x=='' else str(x),
        "WarrantyExpiring": lambda x: '-' if x=='' else str(x),
    },
)
df

,Area,SwitchIP,Hostname,Function,Netmask,Username,Password,Devicetype,Serialno,WarrantyExpiring
0,A,192.168.99.2,S01,Test,255.255.255.0,admin,class,cisco-ios,123879,2019-Dec-23
1,A,192.168.99.3,S01,Web,255.255.255.0,admin,class,cisco-ios,23131,2020-Mar-02
2,B,192.168.99.8,S01,Database,255.255.255.0,admin,class,cisco-ios,313131,-
3,C,192.168.99.13,S01,Devel,255.255.255.0,admin,class,cisco-ios,-,2022-Feb-22
4,C,192.168.99.14,S01,Devel,255.255.255.0,admin,class,cisco-ios,-,2020-Mar-02


Let's compute the remaining warrenty days and stored them into a new column `RemainingWarrentyDays`.  
See [this page](https://docs.python.org/3.8/library/datetime.html#strftime-and-strptime-format-codes) on format codes.

In [53]:
def ComputeRemainingWarrentyDat(value):
    from datetime import datetime
    now = datetime.now()
    dt = now if value == '-' else datetime.strptime(str(value), '%Y-%b-%d')
    return (dt-now).days

df['RemainingWarrentyDay'] = df['WarrantyExpiring'].apply(ComputeRemainingWarrentyDat)
df

,Area,SwitchIP,Hostname,Function,Netmask,Username,Password,Devicetype,Serialno,WarrantyExpiring,RemainingWarrentyDay
0,A,192.168.99.2,S01,Test,255.255.255.0,admin,class,cisco-ios,123879,2019-Dec-23,26
1,A,192.168.99.3,S01,Web,255.255.255.0,admin,class,cisco-ios,23131,2020-Mar-02,96
2,B,192.168.99.8,S01,Database,255.255.255.0,admin,class,cisco-ios,313131,-,0
3,C,192.168.99.13,S01,Devel,255.255.255.0,admin,class,cisco-ios,-,2022-Feb-22,818
4,C,192.168.99.14,S01,Devel,255.255.255.0,admin,class,cisco-ios,-,2020-Mar-02,96


Let's reorder the columns.

In [54]:
df = df[[
    'SwitchIP',
    'Netmask',
    'Area',
    'Hostname',
    'Username',
    'Password',
    'Devicetype',
    'RemainingWarrentyDay',
    'Serialno'
]]

df

,SwitchIP,Netmask,Area,Hostname,Username,Password,Devicetype,RemainingWarrentyDay,Serialno
0,192.168.99.2,255.255.255.0,A,S01,admin,class,cisco-ios,26,123879
1,192.168.99.3,255.255.255.0,A,S01,admin,class,cisco-ios,96,23131
2,192.168.99.8,255.255.255.0,B,S01,admin,class,cisco-ios,0,313131
3,192.168.99.13,255.255.255.0,C,S01,admin,class,cisco-ios,818,-
4,192.168.99.14,255.255.255.0,C,S01,admin,class,cisco-ios,96,-


We finally decide not to use `Area` and `Serialno` columns, so we will drop the column in place.

In [55]:
df = df.drop(
    labels=['Area', 'Serialno'],
    axis=1
)
df

,SwitchIP,Netmask,Hostname,Username,Password,Devicetype,RemainingWarrentyDay
0,192.168.99.2,255.255.255.0,S01,admin,class,cisco-ios,26
1,192.168.99.3,255.255.255.0,S01,admin,class,cisco-ios,96
2,192.168.99.8,255.255.255.0,S01,admin,class,cisco-ios,0
3,192.168.99.13,255.255.255.0,S01,admin,class,cisco-ios,818
4,192.168.99.14,255.255.255.0,S01,admin,class,cisco-ios,96


Let's store the update data into a new Excel file.

In [56]:
with pd.ExcelWriter('Database2.xlsx') as writer:
    df.to_excel(
        writer,
        sheet_name='New',
        index=False
    )

Let's explore other Pandas functions

In [57]:
df

,SwitchIP,Netmask,Hostname,Username,Password,Devicetype,RemainingWarrentyDay
0,192.168.99.2,255.255.255.0,S01,admin,class,cisco-ios,26
1,192.168.99.3,255.255.255.0,S01,admin,class,cisco-ios,96
2,192.168.99.8,255.255.255.0,S01,admin,class,cisco-ios,0
3,192.168.99.13,255.255.255.0,S01,admin,class,cisco-ios,818
4,192.168.99.14,255.255.255.0,S01,admin,class,cisco-ios,96


The column with `0`, `1`, `2`, `3` are indexes to access individual rows of the DataFrame `df`.

In [59]:
# Display rows from index=2 onwards
df[2:]

,SwitchIP,Netmask,Hostname,Username,Password,Devicetype,RemainingWarrentyDay
2,192.168.99.8,255.255.255.0,S01,admin,class,cisco-ios,0
3,192.168.99.13,255.255.255.0,S01,admin,class,cisco-ios,818
4,192.168.99.14,255.255.255.0,S01,admin,class,cisco-ios,96


Retrieve disjoined rows

In [60]:
df.loc[[0,2,4]]

,SwitchIP,Netmask,Hostname,Username,Password,Devicetype,RemainingWarrentyDay
0,192.168.99.2,255.255.255.0,S01,admin,class,cisco-ios,26
2,192.168.99.8,255.255.255.0,S01,admin,class,cisco-ios,0
4,192.168.99.14,255.255.255.0,S01,admin,class,cisco-ios,96


Retrieve disjoined rows and particular columns

In [61]:
df.loc[[0,2,4],['SwitchIP', 'Username', 'Password', 'Devicetype']]

,SwitchIP,Username,Password,Devicetype
0,192.168.99.2,admin,class,cisco-ios
2,192.168.99.8,admin,class,cisco-ios
4,192.168.99.14,admin,class,cisco-ios


Let's retrieve the row with `SwitchIP` equal to `192.168.99.2`

In [62]:
df[df['SwitchIP'] == '192.168.99.2']

,SwitchIP,Netmask,Hostname,Username,Password,Devicetype,RemainingWarrentyDay
0,192.168.99.2,255.255.255.0,S01,admin,class,cisco-ios,26


Let's retrieve the rows with `RemainingWarrentyDay` less than 30 days

In [65]:
df[df['RemainingWarrentyDay'] < 30]

,SwitchIP,Netmask,Hostname,Username,Password,Devicetype,RemainingWarrentyDay
0,192.168.99.2,255.255.255.0,S01,admin,class,cisco-ios,26
2,192.168.99.8,255.255.255.0,S01,admin,class,cisco-ios,0


Let's sort the DataFrame with `RemainingWarrentyDay` (in decending order) followed by `SwitchIP` (in ascending order).

In [39]:
df.sort_values(
    by=['RemainingWarrentyDay', 'SwitchIP'],
    ascending=[False, True]
)

,SwitchIP,Netmask,Hostname,Username,Password,Devicetype,RemainingWarrentyDay
3,192.168.99.13,255.255.255.0,S01,admin,class,cisco-ios,818
4,192.168.99.14,255.255.255.0,S01,admin,class,cisco-ios,96
1,192.168.99.3,255.255.255.0,S01,admin,class,cisco-ios,96
0,192.168.99.2,255.255.255.0,S01,admin,class,cisco-ios,26
2,192.168.99.8,255.255.255.0,S01,admin,class,cisco-ios,0


Retrieve the first two rows that was sorted using last criteria.

In [44]:
df.sort_values(
    by=['RemainingWarrentyDay', 'SwitchIP'],
    ascending=[False, True]
)[0:2]

,SwitchIP,Netmask,Hostname,Username,Password,Devicetype,RemainingWarrentyDay
3,192.168.99.13,255.255.255.0,S01,admin,class,cisco-ios,818
4,192.168.99.14,255.255.255.0,S01,admin,class,cisco-ios,96


We could use `DataFrame.iloc[]` to retrieve by location (not index)

In [66]:
df.sort_values(
    by=['RemainingWarrentyDay', 'SwitchIP'],
    ascending=[False, True]
).iloc[0:2]

,SwitchIP,Netmask,Hostname,Username,Password,Devicetype,RemainingWarrentyDay
3,192.168.99.13,255.255.255.0,S01,admin,class,cisco-ios,818
4,192.168.99.14,255.255.255.0,S01,admin,class,cisco-ios,96


Since `SwitchIP` is unique here, we could use it as index. Let's make `SwitchIP` as index.

In [67]:
df = df.set_index('SwitchIP')
df

,Netmask,Hostname,Username,Password,Devicetype,RemainingWarrentyDay
SwitchIP,,,,,,
192.168.99.2,255.255.255.0,S01,admin,class,cisco-ios,26
192.168.99.3,255.255.255.0,S01,admin,class,cisco-ios,96
192.168.99.8,255.255.255.0,S01,admin,class,cisco-ios,0
192.168.99.13,255.255.255.0,S01,admin,class,cisco-ios,818
192.168.99.14,255.255.255.0,S01,admin,class,cisco-ios,96


Let's retrieve the row with IP address `192.168.99.3`.

In [68]:
df.loc['192.168.99.3']

Netmask                 255.255.255.0
Hostname                          S01
Username                        admin
Password                        class
Devicetype                  cisco-ios
RemainingWarrentyDay               96
Name: 192.168.99.3, dtype: object

Let's retrieve the rows with IP address from `192.168.99.2` to `192.168.99.8`

In [70]:
df.loc['192.168.99.2':'192.168.99.8']

,Netmask,Hostname,Username,Password,Devicetype,RemainingWarrentyDay
SwitchIP,,,,,,
192.168.99.2,255.255.255.0,S01,admin,class,cisco-ios,26
192.168.99.3,255.255.255.0,S01,admin,class,cisco-ios,96
192.168.99.8,255.255.255.0,S01,admin,class,cisco-ios,0


Let's loop through each row

In [72]:
for r in df.itertuples():
    print(f'{r.Hostname}, {r.Username}, {r.Password}')

S01, admin, class
S01, admin, class
S01, admin, class
S01, admin, class
S01, admin, class
